In [ ]:
!pip install transformers farasapy

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 53.0 MB/s 
     |████████████████████████████████| 6.5 MB 39.2 MB/s 
     |████████████████████████████████| 67 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoModelForQuestionAnswering
models_path =[
              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa_4/checkpoint-2960",
              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa_4/checkpoint-1480",
              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa_3/checkpoint-2480",
              
              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa_3/checkpoint-4960",
              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa_2/checkpoint-740" ,
              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa_2/checkpoint-1480",

              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa4/checkpoint-1480" ,
              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa4/checkpoint-2960" ,
              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa/checkpoint-1480" ,

              "/content/drive/MyDrive/Quran_QA/models/quranBERTqa/checkpoint-2960",
              "/content/drive/MyDrive/Quran_QA/models/2/checkpoint-9672" ,
              "/content/drive/MyDrive/Quran_QA/models/2/checkpoint-4836" , 

              "/content/drive/MyDrive/Quran_QA/models/1quranqa4epoch" ,
              "/content/drive/MyDrive/Quran_QA/models/1quranqa2epoch8batch" ,
              "/content/drive/MyDrive/Quran_QA/models/1",

              "/content/drive/MyDrive/Quran_QA/models/1quranqa2epoch",
              "/content/drive/MyDrive/Quran_QA/models/2/checkpoint-2418" ,
              "/content/drive/MyDrive/Quran_QA/models/miniquranqa" , 
              "/content/drive/MyDrive/Quran_QA/saved_model/0",
              "/content/drive/MyDrive/Quran_QA/saved_model/1",
              "/content/drive/MyDrive/Quran_QA/saved_model/2"

]
models =[]
for path in models_path :
  try :
    models.append( AutoModelForQuestionAnswering.from_pretrained(path) )
  except :
    print(path)

# for model in models : model.to("cuda")
  

/content/drive/MyDrive/Quran_QA/models/quranBERTqa_4/checkpoint-2960
/content/drive/MyDrive/Quran_QA/models/quranBERTqa_3/checkpoint-2480
/content/drive/MyDrive/Quran_QA/models/quranBERTqa_3/checkpoint-4960
/content/drive/MyDrive/Quran_QA/models/quranBERTqa_2/checkpoint-1480
/content/drive/MyDrive/Quran_QA/models/quranBERTqa/checkpoint-2960


In [ ]:
len(models)

16

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer , AutoTokenizer
model_path= "/content/drive/MyDrive/Quran_QA/models/quranBERTqa_2/checkpoint-740"
model_path= "/content/drive/MyDrive/Quran_QA/models/quranBERTqa_3/checkpoint-4960"
model_path = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForQuestionAnswering.from_pretrained(model_path)
# model.to("cuda")

In [ ]:
from transformers import pipeline
from scipy.special import softmax
import torch
from collections import Counter
min_answer_length=3
number_of_required_answers = 5
def get_ids_from_logits(outputs):
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    # answer_start = torch.argmax(answer_start_scores)
    answer_starts_probs = softmax(torch.topk(answer_start_scores , 5).values.cpu().detach().numpy())[0]
    # print(answer_starts_probs)
    answer_starts =  torch.topk(answer_start_scores , 5).indices
    # Get the most likely end of answer with the argmax of the score
    # answer_end = torch.argmax(answer_end_scores) + 1
    answer_ends_probs = softmax(torch.topk(answer_end_scores, 5).values.cpu().detach().numpy())[0]
    answer_ends = torch.topk(answer_end_scores, 5).indices +1
    
    return answer_starts.tolist() , answer_ends.tolist() 
def quran_qa(text , question, show_all=False):
    ranked_answers=[]
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")#.to("cuda")
    input_ids = inputs["input_ids"].tolist()[0]
    starts =[]
    ends =[]
    # print(inputs)
    for model in models :
      try :
        outputs = model(**inputs) # predict two lists ( startsss , endsss ) argmax [3 , 4, 5 , 6 ,-4 , 3]
        model_starts , model_ends = get_ids_from_logits(outputs)
        # print(model_starts)
        starts.extend(model_starts[0])
        ends.extend(model_ends[0])
      except BaseException as e :

        print("failed ..", e)
    
    starts =Counter(starts)
    ends =Counter(ends)
    # print(starts)
    # print(ends)
    answer_starts =list(starts.keys())[:number_of_required_answers]
    answer_ends =list(ends.keys())[:number_of_required_answers]
    answer_starts_probs = softmax([starts[x] for x in answer_starts])
    answer_ends_probs = softmax([ends[x] for x in answer_ends ])
    # print(outputs)
    # print(f"Question: {question}")
    idx =0
    for i , answer_start in enumerate(answer_starts):
     for j , answer_end in enumerate(answer_ends):
        idx+=1
        if (
                answer_end < answer_start
                or answer_end - answer_start + 1 < min_answer_length
            ):
                continue
        answer = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
        )
        
        ranked_answers.append(
            {
                'answer': answer,
                  'rank' : len(ranked_answers)+1,
                  'score':float(answer_starts_probs[i]*answer_ends_probs[j])
            }
        )
    #sort by probability 
    ranked_answers.sort(key =lambda x : x['score'], reverse =True)
    ranked_answers = ranked_answers[:number_of_required_answers]
    # reset rank 
    for i , answer in enumerate(ranked_answers): answer['rank']=i+1
    if len(ranked_answers)==0: raise BaseException("empty list ")
    # print('top predicted answer:', )
    return ranked_answers

In [ ]:
text ="ولا تقولن لشئ إني فاعل ذلك غدا إلا أن يشاء الله . واذكر ربك إذا نسيت وقل عسى أن يهديني ربي لأقرب من هذا رشدا . ولبثوا في كهفهم ثلاث مائة سنين وازدادوا تسعا قل الله أعلم بما لبثوا له غيب السماوات والأرض ابصر به واسمع مالهم من دونه من ولي ولا يشرك في حكمه احدا ."
q="كم لبث اصحاب الكهف في الكهف ؟"
quran_qa(text ,q)

failed .. index out of range in self


[{'answer': 'ثلاث مائة سنين وازدادوا تسعا',
  'rank': 1,
  'score': 0.11372808073212853},
 {'answer': 'ثلاث مائة سنين', 'rank': 2, 'score': 0.11372808073212853},
 {'answer': 'مائة سنين وازدادوا تسعا',
  'rank': 3,
  'score': 0.11372808073212853},
 {'answer': 'مائة سنين', 'rank': 4, 'score': 0.11372808073212853},
 {'answer': 'ولبثوا في كهفهم ثلاث مائة سنين وازدادوا تسعا',
  'rank': 5,
  'score': 0.11372808073212853}]

In [ ]:
train_data_path = r'/content/drive/MyDrive/Quran_QA/Q_QA_task_original_code/quranqa/datasets/qrcd_v1.1_train.jsonl'
AQQAC_data = r'/content/drive/MyDrive/Quran_QA/data/AQQAC_less_384.jsonl'
dev_data_path = r'/content/drive/MyDrive/Quran_QA/Q_QA_task_original_code/quranqa/datasets/qrcd_v1.1_dev.jsonl'

import json 
def read_data(datapath):
  with open(datapath ,'rb') as fp:
      datalist = list(fp)
  data =[]
  for json_str in datalist:
      result = json.loads(json_str)
      # print(f"result: {result}")
      data.append(result)
  return data 
val_data = read_data(dev_data_path)

In [ ]:
testset_path ="/content/drive/MyDrive/Quran_QA/Q_QA_task_original_code/quranqa/datasets/qrcd_v1.1_test_noAnswers.jsonl"
testset_data = read_data(testset_path)

In [ ]:
result ={}
for sample in testset_data :
  result[sample['pq_id']]=quran_qa(sample['passage'], sample['question'])

In [ ]:
import json 
with open('star_run13.json', 'w' , encoding= 'utf8') as fp:
  json.dump(result , fp, ensure_ascii=False)

In [ ]:
## 